# ✉️ Messages
# ✉️ 메시지
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

메시지는 LangChain에서 모델의 컨텍스트를 구성하는 기본 단위입니다. 메시지는 모델의 입력과 출력을 나타내며, LLM과 상호작용할 때 대화 상태를 표현하는 데 필요한 콘텐츠와 메타데이터를 모두 담고 있습니다.

## Setup
## 설정

Load and/or check for needed environmental variables

필요한 환경 변수를 로드하거나 확인합니다

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=<not set>
LANGSMITH_TRACING=<not set>
LANGSMITH_PROJECT=<not set>


## Human👨‍💻 and AI 🤖 Messages
## Human👨‍💻 메시지와 AI 🤖 메시지

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-5-nano", 
    system_prompt="You are a full-stack comedian"
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [4]:
print(result["messages"][-1].content)

Hey there! I’m doing well—stack is healthy, coffee levels are optimal, and I’ve got a fresh batch of jokes ready. How are you?

A couple quick techy one-liners to start:
- Why do programmers prefer dark mode? Because light attracts bugs.
- I would tell you a UDP joke, but you might not get it.
- My code runs on coffee and sarcasm—mostly sarcasm.

What can I help you with today? Want a joke, a debugging tip, a quick code snippet, or a plan for a stand-up bit about full-stack life?


In [5]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m doing well—stack is healthy, coffee levels are optimal, and I’ve got a fresh batch of jokes ready. How are you?

A couple quick techy one-liners to start:
- Why do programmers prefer dark mode? Because light attracts bugs.
- I would tell you a UDP joke, but you might not get it.
- My code runs on coffee and sarcasm—mostly sarcasm.

What can I help you with today? Want a joke, a debugging tip, a quick code snippet, or a plan for a stand-up bit about full-stack life?



### Altenative formats
### 대체 형식
#### Strings
#### 문자열
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

LangChain이 컨텍스트에서 역할을 추론할 수 있는 경우가 있으며, 이때는 간단한 문자열만으로도 메시지를 생성할 수 있습니다.

In [7]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [8]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Baseball: chalked diamond, sun on the infield.
The pitcher winds; the ball answers with heat.
The bat cracks; silence shatters, the crowd breathes.
Leather speaks; gloves swallow ground.
Three strikes, the inning exhales and dies.
Four balls—walk to first, a quiet advance.
A stolen base—a blur of grass and steel.
Nine innings, time stitched in dirt and hope.
Rain delays, legends, late-inning miracles.
Seventh-inning stretch; hats rise to the anthem.

Baseball endures, old dream, fresh memory.


#### Dictionaries
#### 딕셔너리

In [9]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Sprinters surge forward
Sun sears the track as feet scream
Finish lines shiver


There are multiple roles:

여러 역할이 있습니다:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
## 출력 형식
### messages
### 메시지
Let's create a tool so agent will create some tool messages.

에이전트가 도구 메시지를 생성하도록 도구를 만들어 봅시다.

In [10]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [11]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [12]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


In [13]:
result["messages"][-1].content

'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'

In [14]:
print(len(result["messages"]))

4


In [15]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_uOLf0ZxcBTNBT2a77IZJi1jO)
 Call ID: call_uOLf0ZxcBTNBT2a77IZJi1jO
  Args:
    text: Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


### Other useful information
### 기타 유용한 정보
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

위에서 출력된 메시지들은 messages 리스트에 저장된 정보의 일부만 선택한 것입니다. 사용 가능한 모든 정보를 자세히 살펴봅시다!

In [16]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='13b18ae4-f92e-4582-925a-39d7984806a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 555, 'prompt_tokens': 170, 'total_tokens': 725, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbI0V3W0uPCDlcnD4GYTydYxCsAJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0055426a-ee06-4daa-a36c-4074fa9c1ceb-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'}, 'id': 'call_uOLf0ZxcBTNBT2a77IZJi1jO', 'type': 'tool_call'}], usage_

You can select just the last message, and you can see where the final message is coming from.

마지막 메시지만 선택할 수 있으며, 최종 메시지가 어디서 왔는지 확인할 수 있습니다.

In [17]:
result["messages"][-1]

AIMessage(content='Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 231, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--61eb6a33-1bda-46d7-8d00-f3aaea2487a3-0', usage_metadata={'input_tokens': 231, 'output_tokens': 21, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
result["messages"][-1].usage_metadata

{'input_tokens': 231,
 'output_tokens': 21,
 'total_tokens': 252,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [19]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
### 직접 해보세요!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

시스템 프롬프트를 변경하고, `pretty_printer`를 사용해 메시지를 출력하거나 `results`를 직접 탐색해 보세요. Human, AI, Tool 메시지와 관련 메타데이터를 확인해 보세요. 최종 결과가 에이전트 활동의 전체 히스토리를 어떻게 제공하는지 주목하세요!

In [20]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()